Project Statement
============
As with Project 1, apply the ideas of ch. 1 - 3 as appropriate.
Develop and demonstrate your capabilities with:
 * Regression (ch. 4)
 * Support Vector Machines (SVM's) (ch. 5)
 
 
 Starting Point
 --------------
 To start this project I'm going to pull from the Titanic dataset part of the previous, since I was a little disappointed that Derek Byrnes got a higher score than me. Friendly rivalry and what not. :) So now that we're working together, I'll see if I can pull some of his techniques in to produce a better score.

In [3]:
import pandas

raw_training = pandas.read_csv("titanic/train.csv")
raw_test = pandas.read_csv("titanic/test.csv")
example_output = pandas.read_csv("titanic/gender_submission.csv")

y = raw_training["Survived"].copy()
X = raw_training.drop("Survived", axis=1)
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [4]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
import re

class RegexTransform(BaseEstimator, TransformerMixin):
    def __init__(self, regex=".*", groupNum=0):
        self.regex = re.compile(regex)
        self.groupNum = groupNum
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.applymap((lambda x: RegexTransform.applyRegex(x, self)))
    
    @staticmethod
    def applyRegex(val, self):
        match = self.regex.search(val)
        newVal = ""
        if (match):
            newVal = match.group(groupNum)
        if (newVal is None):
            newVal = ""
        return newVal

regexTest = RegexTransform(regex="(.*?),", groupNum=1)
results = regexTest.fit_transform(X)

TypeError: ('expected string or bytes-like object', 'occurred at index PassengerId')

In [ ]:
import sklearn.compose
import sklearn.impute
import sklearn.pipeline
import sklearn.preprocessing

class TitanicDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_scale=true, numeric_nBins=5, numeric_encode=ordinal, numeric_strategy=quantile):
        numeric_steps=[
            ('imputer', SimpleImputer()),
            ('scaler', StandardScaler()),
            ('discretizer', KBinsDiscretizer(n_bins=discretize_numeric_nbins,encode=numeric_encode,strategy=numeric_strategy))
        ]
        self.numeric_pipeline = Pipeline(numeric_steps)

        self.categorical_pipeline = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(sparse=False, handle_unknown ='ignore'))
        ])

        self.namesPipeline = Pipeline(steps=[
            ('lastName', RegexTransform(regex="(.*?),", groupNum=1)),
            ('oneHot', OneHotEncoder(sparse=False, handle_unknown ='ignore'))
        ])

        self.column_transform = ColumnTransformer(sparse_threshold=0,transformers=[
            ('numerical', self.numeric_pipeline, ["Age", "Fare"]),
            ('categorical', self.categorical_pipeline, ["Sex", "Embarked", "Pclass"]),
            ('passthrough', "passthrough", ["SibSp", "Parch"]),
            ('names', self.namesPipeline, ["Name"])
        ])
    
    def fit(self, data, labels=None):
        self.column_transform.fit(data, labels)
        
        self.features = ["Age", "Fare" ]
        name, pipeline, columns = self.column_transform.transformers_[1]
        name, transform = pipeline.steps[-1]
        self.features += getOneHotEncoderColumns(transform, columns)
        self.features += ["SibSp", "Parch"]
        name, pipeline, columns = self.column_transform.transformers_[3] 
        name, transform = pipeline.steps[-1]
        self.features += getOneHotEncoderColumns(transform, columns)
        return self
    
    def transform(self, data):
        tmpData = self.column_transform.transform(data)

        newNames = ["Age", "Fare" ]
        name, pipeline, columns = self.column_transform.transformers_[1]
        name, transform = pipeline.steps[-1]
        newNames += getOneHotEncoderColumns(transform, columns)
        newNames += ["SibSp", "Parch"]
        name, pipeline, columns = self.column_transform.transformers_[3] 
        name, transform = pipeline.steps[-1]
        newNames += getOneHotEncoderColumns(transform, columns)
        
        df = pandas.DataFrame(tmpData, columns=newNames)
        #dummies = pandas.DataFrame(columns=self.features)
        #return pandas.concat((df, dummies)).fillna(0)
        return df

titanicTransformer = TitanicDataTransformer()
clean_training = titanicTransformer.fit_transform(X)

print(clean_training.head())
print(trainingWithNames.head())